In [3]:
import numpy as np
import pandas as pd
import ccxt
import csv
import json
import datetime
import backtrader as bt
from pathlib import Path

exchange = ccxt.ftx({
    'enableRateLimit': True,
    'options': {
        'adjustForTimeDifference': True
    }
})
_ =exchange.loadMarkets()
milliseconds = {key: int(value) * 1000 for key, value in exchange.timeframes.items()}


In [6]:
def retryFetchOHLCV(exchange, max_retries, symbol, timeframe, since):
    numRetries = 0
    OHLCV = None
    while OHLCV is None:
        try:
            numRetries += 1
            OHLCV = exchange.fetchOHLCV(symbol, timeframe, since, 700)
            return OHLCV
        except Exception as e:
            if numRetries > max_retries:
                raise e

def getCandles(exchange, maxRetries, symbol, timeframe, fromDatetime, toDatetime):
    since = fromDatetime.timestamp() * 1000
    to = toDatetime.timestamp() * 1000
    candles = []
    last = milliseconds[timeframe]
    while since < to:
        OHLCV = retryFetchOHLCV(exchange, maxRetries, symbol, timeframe, since)
        print('%s candles obtained' % (len(OHLCV)))
        if not len(OHLCV):
            break
        candles += OHLCV
        since = candles[-1][0]
        if since >= (to - last):
            break
    return candles

def writeToCSV(filename, data):
    p = Path('../data/')
    p.mkdir(parents=True, exist_ok=True)
    fullPath = p / filename
    with fullPath.open('w', newline='') as f:
        fieldNames = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        csvWriter = csv.writer(f)
        csvWriter.writerow(fieldNames)
        csvWriter.writerows(data)

In [7]:
delta = datetime.timedelta(days=1)
fromDateTime = datetime.datetime.now() - delta
toDateTime = datetime.datetime.now()

candleData = getCandles(exchange, 3, 'BTC-PERP', '5m', fromDateTime, toDateTime)
writeToCSV('FTX-BTCPERP-12w-5m.csv', candleData)

288 candles obtained
